# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.123236e+02     1.555433e+00
 * time: 0.3784139156341553
     1     1.082452e+01     8.737443e-01
 * time: 2.4346070289611816
     2    -1.166514e+01     9.603391e-01
 * time: 3.108814001083374
     3    -3.454761e+01     7.816577e-01
 * time: 4.1425909996032715
     4    -4.822310e+01     5.659699e-01
 * time: 5.173605918884277
     5    -5.716133e+01     2.014148e-01
 * time: 6.16612696647644
     6    -5.980823e+01     1.246975e-01
 * time: 6.8331029415130615
     7    -6.086007e+01     5.984695e-02
 * time: 7.502481937408447
     8    -6.128802e+01     5.404932e-02
 * time: 8.189166069030762
     9    -6.156802e+01     4.789208e-02
 * time: 8.875817060470581
    10    -6.178719e+01     2.952494e-02
 * time: 9.549713850021362
    11    -6.197393e+01     2.834982e-02
 * time: 10.21753191947937
    12    -6.204600e+01     2.417196e-02
 * time: 10.90040898323059
    13    -6.211137e+01     1.786655e-02
 * time: 11.57434606552124
    

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671054
    AtomicLocal         -18.8557669
    AtomicNonlocal      14.8522657
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485359 
    Xc                  -19.3336815

    total               -62.261666459318
